### Implementing a Data Loader

In [1]:
# For the efficient data loader implementation,we will use PyTorch's built-in Dataset and DataLoaders classes.

In [2]:
# STEPS:
    #1) Tokenize the entire data.
    #2) Use a sliding window to chunk the book into overlapping sequences of max_length.
    #3) Return the total no. of rows in the4 dataset.
    #4) Return a single row from the dataset.

In [3]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())


2.8.0+cpu
CUDA available: False


In [12]:
from torch.utils.data import Dataset,DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        self.input_ids = []
        self.target_ids = []

        #Tokenize the entire text 
        token_ids = tokenizer.encode(txt,allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequennces of max_length
        for i in range(0,len(token_ids) - max_length, stride):
             input_chunk = token_ids[i:i+max_length]
             target_chunk = token_ids[i+1:i+max_length+1]
             self.input_ids.append(torch.tensor(input_chunk))
             self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self,idx):
        return self.input_ids[idx],self.target_ids[idx]

In [13]:
# The GPTDatasetV1 is listing 2.5 based on the PyTorch Dataset classes.
# It define how individual rows are fetched from the dataset.

# GPTDatasetV1 is used to load the input in batches via Pytorch Dataloader

In [14]:
# STEPS:
    #1) Initialize the Tokenizer.
    #2) create Dataset
    #3) drop_last = True drop the last batch if it shorter than specified batch_size to prevent loss spikes during training.
    #4) The no. of CPU processes to use for preprocessing

In [15]:
 import importlib
import tiktoken

print("tiktoken version:",importlib.metadata.version("tiktoken"))

def create_dataloader_v1(txt,batch_size = 4,max_length = 256,stride = 128 , shuffle = True , drop_last = True , num_workers=0):

    #Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")  #BPE

    #create dataset
    dataset = GPTDatasetV1(txt,tokenizer,max_length,stride)

    #create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = shuffle,
        drop_last = drop_last,
        num_workers = num_workers
    )

    return dataloader

tiktoken version: 0.12.0


# Test the dataloader

In [16]:
with open("C:/Users/sumit/Downloads/the-verdict.txt","r",encoding = "utf-8") as f:
    raw_text=f.read()

In [17]:
# Convert dataloader into python iterator to fetch the next entry via python's vuilt-in next() function.

In [18]:
import torch
print("pyTorch version:",torch.__version__)
dataloader = create_dataloader_v1(
    raw_text,batch_size = 1, max_length=4 , stride=1 , shuffle = False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

pyTorch version: 2.8.0+cpu
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [19]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [21]:
# If we compare the first with the second batch, we can see that the second batch's token IDs are shifted by one postion compared to the first batch.
# in Deep Learning, small batch sizes required less memory during training but lead to more noisy model 

In [22]:
dataloader = create_dataloader_v1(raw_text,batch_size = 8 , max_length = 4,stride = 4 , shuffle = False)

data_iter = iter(dataloader)
inputs,targets = next(data_iter)
print("Inputs:\n",inputs)
print("\nTargets:\n",targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
